In [2]:
!pip install ultralytics
!pip install roboflow
import zipfile
import os

zip_path = '/content/Aerial Maritime Dataset.zip'  # Replace with your file path
extract_path = '/content/dataset'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Dataset extracted to:", extract_path)


Dataset extracted to: /content/dataset


In [6]:
import zipfile
import os

# Path to the uploaded zip file
annotated_zip_path = '/content/annotated_dataset.zip'  # Update path if needed
annotated_dataset_path = '/content/annotated_dataset/'

# Extract the zip file
with zipfile.ZipFile(annotated_zip_path, 'r') as zip_ref:
    zip_ref.extractall(annotated_dataset_path)

# List the extracted folders and files to verify the structure
dataset_structure = {
    "root": os.listdir(annotated_dataset_path),
    "train": os.listdir(os.path.join(annotated_dataset_path, 'train')) if 'train' in os.listdir(annotated_dataset_path) else None,
    "valid": os.listdir(os.path.join(annotated_dataset_path, 'valid')) if 'valid' in os.listdir(annotated_dataset_path) else None,
    "test": os.listdir(os.path.join(annotated_dataset_path, 'test')) if 'test' in os.listdir(annotated_dataset_path) else None,
}

dataset_structure


{'root': ['data.yaml', 'valid', 'README.roboflow.txt', 'test', 'train'],
 'train': ['labels', 'images'],
 'valid': ['labels', 'images'],
 'test': ['labels', 'images']}

In [8]:
import yaml

# Load the data.yaml file
data_yaml_path = '/content/annotated_dataset/data.yaml'
with open(data_yaml_path, 'r') as file:
    data_info = yaml.safe_load(file)

# Print the content of the data.yaml file
data_info


{'train': '../train/images',
 'val': '../valid/images',
 'test': '../test/images',
 'nc': 1,
 'names': ['boat-car-dock-jetski-lift'],
 'roboflow': {'workspace': 'mahaveer',
  'project': 'aerial-maritime-ivd87',
  'version': 1,
  'license': 'Private',
  'url': 'https://app.roboflow.com/mahaveer/aerial-maritime-ivd87/1'}}

In [ ]:
from ultralytics import YOLO

# Initialize the YOLO model (YOLOv8 small for faster training)
model = YOLO('yolov8n.pt')  # YOLO Nano model (small and quick for testing)

# Train the model
model.train(data=data_yaml_path, epochs=50, imgsz=640, batch=16)


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 6.25M/6.25M [00:00<00:00, 90.3MB/s]


Ultralytics 8.3.41 🚀 Python-3.10.12 torch-2.5.1+cu121 CPU (Intel Xeon 2.20GHz)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/annotated_dataset/data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, sho

100%|██████████| 755k/755k [00:00<00:00, 28.3MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

train: Scanning /content/annotated_dataset/train/labels... 32 images, 0 backgrounds, 0 corrupt: 100%|██████████| 32/32 [00:00<00:00, 252.30it/s]

train: New cache created: /content/annotated_dataset/train/labels.cache


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


In [ ]:
# Evaluate the model on the validation dataset
metrics = model.val()
print(metrics)
# Run inference on test images
results = model.predict('/content/annotated_dataset/test/images', save=True)


In [ ]:
# Save the trained model (best weights)
model.best.save('/content/YOLO_model')
# Run inference on test images
results = model.predict('/content/annotated_dataset/test/images', save=True)

# Display results
results.show()  # Show the output images with detected objects


In [ ]:
# Check the output directory where results are saved
import os
output_dir = '/content/runs/detect/exp'  # Default output path for detection results
output_images = os.listdir(output_dir)

# Display some output images
from IPython.display import Image
Image(filename=os.path.join(output_dir, output_images[0]))  # Show first image
print("mAP Score:", metrics['mAP_0.5'])  # Precision at IoU threshold 0.5
print("Precision:", metrics['precision'])
print("Recall:", metrics['recall'])


In [ ]:
# Re-train with merged class
model.train(data='/content/annotated_dataset_modified/data.yaml', epochs=50, imgsz=640, batch=16)
# Evaluate the original model
original_metrics = model.val(data='/content/annotated_dataset/data.yaml')
print("Original Model Metrics:", original_metrics)

# Evaluate the modified model (after merging classes)
modified_model = YOLO('/content/YOLO_model')  # Load the modified model
modified_metrics = modified_model.val(data='/content/annotated_dataset_modified/data.yaml')
print("Modified Model Metrics:", modified_metrics)
# Save results directory to Google Drive or download manually
shutil.move('/content/runs/detect/exp', '/content/YOLO_results')
